# Find district alias keywords

Data source: [Danh sách các đơn vị hành chính cấp huyện không còn tồn tại](https://vi.wikipedia.org/wiki/Danh_s%C3%A1ch_%C4%91%C6%A1n_v%E1%BB%8B_h%C3%A0nh_ch%C3%ADnh_c%E1%BA%A5p_huy%E1%BB%87n_c%E1%BB%A7a_Vi%E1%BB%87t_Nam#Danh_s%C3%A1ch_c%C3%A1c_%C4%91%C6%A1n_v%E1%BB%8B_h%C3%A0nh_ch%C3%ADnh_c%E1%BA%A5p_huy%E1%BB%87n_kh%C3%B4ng_c%C3%B2n_t%E1%BB%93n_t%E1%BA%A1i)

In [1]:
import pandas as pd
import re
import pickle
from vietadminunits.utils import to_key
from vietadminunits import get_data, parse_address

In [2]:
def check_containing_keyword(district, change):
    if change.count(district) == 2:
        return 1
    else:
        return 0
    
def get_district_key(text):
    au = parse_address(text, 2)
    return au.district_key

def prepare_address(province, change):
    address = re.sub(f'.* thành ', '', change) + ', ' + province
    return address

def get_province_english(text):
    au = parse_address(text, 1)
    return au.province_english

In [3]:
df = pd.read_csv('../../data/input/district_change.csv')

In [4]:
df

,province,old_district,year,change
0,An Giang,Huệ Đức,1977.0,sáp nhập toàn bộ vào huyện Châu Thành
1,An Giang,Bảy Núi,1979.0,chia tách thành huyện Tri Tôn và huyện Tịnh Biên
2,An Giang,Phú Châu,1991.0,chia tách thành huyện An Phú và huyện Tân Châu
3,Bà Rịa – Vũng Tàu,Châu Thành,1994.0,"chia tách thành thị xã Bà Rịa, huyện Tân Thành..."
4,Bà Rịa – Vũng Tàu,Long Đất,2003.0,chia tách thành huyện Long Điền và huyện Đất Đỏ
...,...,...,...,...
132,Vĩnh Long,Cái Nhum,1977.0,sáp nhập với huyện Châu Thành Tây thành huyện ...
133,Vĩnh Long,Châu Thành Tây,1977.0,sáp nhập với huyện Cái Nhum thành huyện Long Hồ
134,Vĩnh Phúc,Yên Lãng,1977.0,sáp nhập với huyện Bình Xuyên thành huyện Mê Linh
135,Vĩnh Phúc,Vĩnh Lạc,1995.0,chia tách thành huyện Vĩnh Tường và huyện Yên Lạc


In [5]:
df.dropna(inplace=True)

In [6]:
df['province_english'] = df.province.apply(get_province_english)
df['old_district_key'] = df.old_district.apply(to_key, args=(2,))
df['containing_keyword_two_times'] = df.apply(lambda x: check_containing_keyword(x.old_district, x.change), axis=1)

## Create alias district_keys

In [7]:
df_change = df[(df.change.fillna('').str.contains('sáp nhập|đổi tên')) & ~(df.change.fillna('').str.contains('một phần'))].copy()

In [8]:
df_change

,province,old_district,year,change,province_english,old_district_key,containing_keyword_two_times
0,An Giang,Huệ Đức,1977.0,sáp nhập toàn bộ vào huyện Châu Thành,An Giang,hueduc,0
5,Bà Rịa – Vũng Tàu,Tân Thành,2018.0,"thành lập thị xã, đổi tên thành thị xã Phú Mỹ",Ba Ria - Vung Tau,tanthanh,0
7,Bắc Kạn,Chợ Rã,1984.0,đổi tên thành huyện Ba Bể,Bac Kan,chora,0
9,Bạc Liêu,Minh Hải,1984.0,đổi tên thành thị xã Bạc Liêu,Bac Lieu,minhhai,0
14,Bình Dương,Châu Thành,1977.0,sáp nhập vào huyện Tân Uyên và thị xã Thủ Dầu Một,Binh Duong,chauthanh,0
15,Bình Dương,Lái Thiêu,1977.0,sáp nhập với huyện Dĩ An thành huyện Thuận An,Binh Duong,laithieu,0
18,Bình Phước,Phước Bình,1977.0,sáp nhập với huyện Bù Đăng và huyện Bù Đốp thà...,Binh Phuoc,phuocbinh,0
23,Cà Mau,Ngọc Hiển,1984.0,đổi tên thành huyện Đầm Dơi,Ca Mau,ngochien,0
26,Cao Bằng,Phục Hòa,2020.0,sáp nhập với huyện Quảng Uyên thành huyện Quản...,Cao Bang,phuchoa,0
27,Cao Bằng,Quảng Uyên,2020.0,sáp nhập với huyện Phục Hòa thành huyện Quảng Hòa,Cao Bang,quanguyen,0


In [9]:
# Remove Thủ Đức vì không cần
df_change = df_change[df_change.old_district!='Thủ Đức (quận)'].copy()

In [10]:
df_change['address'] = df_change.apply(lambda row: prepare_address(row.province, row.change), axis=1)
df_change['district_key'] = df_change.address.apply(get_district_key)

After doing research, I found district_english `None` are fine, because they are moved to another province or revert it's changing.

In [11]:
df_change.dropna(inplace=True)

In [12]:
DICT_alias_district_keys = {}
for province_english in df_change['province_english'].unique():
    district_data = {}
    for row in df_change[df_change['province_english']==province_english].itertuples():
        district_data[row.old_district_key] = row.district_key
    DICT_alias_district_keys[province_english] = district_data

In [13]:
DICT_alias_district_keys

{'An Giang': {'hueduc': 'chauthanh'},
 'Ba Ria - Vung Tau': {'tanthanh': 'phumy'},
 'Bac Kan': {'chora': 'babe'},
 'Bac Lieu': {'minhhai': 'baclieu'},
 'Binh Duong': {'chauthanh': 'tanuyen', 'laithieu': 'thuanan'},
 'Binh Phuoc': {'phuocbinh': 'phuoclong'},
 'Ca Mau': {'ngochien': 'damdoi'},
 'Cao Bang': {'phuchoa': 'quanghoa',
  'quanguyen': 'quanghoa',
  'thongnong': 'haquang',
  'tralinh': 'trungkhanh'},
 'Dien Bien': {'laichau': 'muonglay', 'muonglay': 'muongcha'},
 'Dong Nai': {'vinhan': 'vinhcuu'},
 'Dong Thap': {'thanhhung': 'lapvo'},
 'Ha Nam': {'hanam': 'phuly'},
 'Ha Noi': {'daphuc': 'socson', 'kimanh': 'socson'},
 'Hai Phong': {'catba': 'cathai'},
 'Hau Giang': {'mythanh': 'vithanh', 'tanhiep': 'ngabay'},
 'Hoa Binh': {'kyson': 'hoabinh'},
 'Khanh Hoa': {'khanhxuong': 'dienkhanh'},
 'Kien Giang': {'hatien': 'kienluong'},
 'Lao Cai': {'camduong': 'laocai'},
 'Nam Dinh': {'myloc': 'namdinh'},
 'Ninh Binh': {'giakhanh': 'hoalu',
  'hoanglong': 'nhoquan',
  'tamdiep': 'yenmo'},


## Create half district keys

In [14]:
df_split = df[df.containing_keyword_two_times==1].copy()

In [15]:
df_split

,province,old_district,year,change,province_english,old_district_key,containing_keyword_two_times
12,Bến Tre,Mỏ Cày,2009.0,chia tách thành huyện Mỏ Cày Bắc và huyện Mỏ C...,Ben Tre,mocay,1
17,Bình Dương,Tân Uyên,2013.0,chia tách thành thị xã Tân Uyên và huyện Bắc T...,Binh Duong,tanuyen,1
21,Bình Thuận,Hàm Thuận,1982.0,chia tách thành huyện Hàm Thuận Bắc và huyện H...,Binh Thuan,hamthuan,1
45,Hà Nội,Từ Liêm,2013.0,chia tách thành quận Bắc Từ Liêm và Nam Từ Liêm,Ha Noi,tuliem,1
99,Quảng Nam,Trà My,2003.0,chia tách thành huyện Bắc Trà My và huyện Nam ...,Quang Nam,tramy,1
124,Tiền Giang,Gò Công,1979.0,chia tách thành huyện Gò Công Đông và Gò Công Tây,Tien Giang,gocong,1


In [16]:
# This is a particular case that is duplicated district_key
df_split = df_split[df_split.old_district!='Tân Uyên'].copy()

In [17]:
# Use Google Trend to decide new district
df_split.loc[df_split.old_district=='Mỏ Cày', 'default'] = 'mocaynam'
df_split.loc[df_split.old_district=='Hàm Thuận', 'default'] = 'hamthuannam'
df_split.loc[df_split.old_district=='Từ Liêm', 'default'] = 'namtuliem'
df_split.loc[df_split.old_district=='Trà My', 'default'] = 'namtramy'
df_split.loc[df_split.old_district=='Gò Công', 'default'] = 'gocongtay'

In [18]:
df_test = pd.DataFrame(get_data())

In [19]:
df_test['ward_key'] = df_test['short_ward'].apply(to_key, args=(3,))

In [20]:
df_test['district_key'] = df_test['short_district'].apply(to_key, args=(2,))

In [21]:
df_split['old_district_key'] = df_split['old_district'].apply(to_key, args=(2,))

In [22]:
DICT_half_district_keys = {}

for province_english in df_split['province_english'].unique():
    half_district_data = {}
    for half_district_key in df_split[df_split['province_english']==province_english]['old_district_key'].unique():
        district_data = {}
        tmp_df = df_test[(df_test['province_english']==province_english) & (df_test['district_key'].str.contains(half_district_key))].copy()
        for district_key in tmp_df['district_key'].unique():
            tmp_ward_keys = tmp_df[tmp_df['district_key']==district_key]['ward_key'].dropna().tolist()
            district_data[district_key] = tmp_ward_keys
        half_district_data[half_district_key] = {
            'default': df_split[(df_split['province_english']==province_english) & (df_split['old_district_key']==half_district_key)]['default'].values[0],
            'district_keys': district_data
        }
    
    DICT_half_district_keys[province_english] = half_district_data

In [23]:
DICT_half_district_keys

{'Ben Tre': {'mocay': {'default': 'mocaynam',
   'district_keys': {'mocaynam': ['mocay',
     'dinhthuy',
     'daphuochoi',
     'tanhoi',
     'phuochiep',
     'binhkhanh',
     'anthanh',
     'andinh',
     'thanhthoib',
     'tantrung',
     'anthoi',
     'thanhthoia',
     'minhduc',
     'ngaidang',
     'camson',
     'huongmy'],
    'mocaybac': ['phumy',
     'hungkhanhtrunga',
     'thanhtan',
     'thanhngai',
     'tanphutay',
     'phuocmytrung',
     'tanthanhbinh',
     'thanhan',
     'hoaloc',
     'tanthanhtay',
     'tanbinh',
     'nhuanphutan',
     'khanhthanhtan']}}},
 'Binh Thuan': {'hamthuan': {'default': 'hamthuannam',
   'district_keys': {'hamthuanbac': ['malam',
     'phulong',
     'lada',
     'dongtien',
     'thuanhoa',
     'donggiang',
     'hamphu',
     'hongliem',
     'thuanminh',
     'hongson',
     'hamtri',
     'hamduc',
     'hamliem',
     'hamchinh',
     'hamhiep',
     'hamthang',
     'dami'],
    'hamthuannam': ['thuannam',
     'myth

In [55]:
with open('../../vietadminunits/data/parse.pkl', 'rb') as f:
    data = pickle.load(f)
    
    
data['DICT_alias_district_keys'] = DICT_alias_district_keys
data['DICT_half_district_keys'] = DICT_half_district_keys

with open('../../vietadminunits/data/parse.pkl', 'wb') as f:
    pickle.dump(data, f)
    
    
[key for key in data]

['DICT_long_province_alphanumerics',
 'DICT_long_district_alphanumerics',
 'DICT_long_ward_alphanumerics',
 'LIST_safe_long_district_alphanumerics',
 'LIST_safe_long_ward_alphanumerics',
 'LIST_province_keys_1',
 'LIST_province_keys_2',
 'LIST_province_keys_3',
 'DICT_province_map',
 'DICT_district_map',
 'DICT_ward_map',
 'LIST_duplicated_district_province_keys',
 'DICT_duplicated_district_keys',
 'LIST_duplicated_ward_district_keys',
 'DICT_duplicated_ward_keys',
 'DICT_unique_long_district_alphanumerics',
 'DICT_not_unique_long_district_alphanumerics',
 'LIST_contains_province_key_long_district_alphanumerics',
 'DICT_unique_district_keys',
 'DICT_not_unique_district_keys',
 'DICT_alias_province_keys',
 'DICT_alias_district_keys',
 'DICT_half_district_keys']